### Imports 

In [ ]:
# Imports
import torch as pt
from torch import nn

print(f"Torch version: {pt.__version__}")

# if pt.cuda.is_available():
#     device = 'cuda'
# if pt.backends.mps.is_available():
#     device = 'mps'
# else:
#     device= 'cpu'
device = 'cpu'
print(f'device: {device}')

### Downloading a custom dataset

In [ ]:
from pathlib import Path
import importLib
from sys import path
import zipfile


# Create directory
data_path = Path(f"{path[0]}/data")
image_path = data_path / 'pizza_steak_sushi'
if image_path.exists():
    print('Already exists')
else:
    image_path.mkdir(parents=True)


# Download pizza, steak and sushi data
# open skapar en zip fil som sedan fylls genom request
importLib.import_from_github('https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip',directory=data_path)
with zipfile.ZipFile(data_path/'pizza_steak_sushi.zip', 'r') as zip_ref:
    print('Unzipping pizza, steak and sushi data')
    zip_ref.extractall(image_path)
Path.unlink(data_path/'pizza_steak_sushi.zip')


In [ ]:
# Setup training and testing paths
train_dir = image_path / 'train'
test_dir = image_path / 'test'

train_dir, test_dir

### Create dataset and dataloaders

In [ ]:
from torchvision import transforms
simple_transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor()
])

In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets

train_dataset = datasets.ImageFolder(root = train_dir, transform=simple_transform)
test_dataset = datasets.ImageFolder(root = test_dir, transform=simple_transform)

In [ ]:
import os
BATCH_SIZE = 32
NUM_WORKERS = round(os.cpu_count()*(3/4))
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=32,
    num_workers=NUM_WORKERS,
    shuffle=False
)

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=32,
    num_workers=NUM_WORKERS,
    shuffle=False
)

### Create model

In [29]:
from torch import nn

class ModelWithoutAugmentation(nn.Module):
    def __init__(self, input_features:int,output_features:int, hidden_units:int=10):
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(input_features, hidden_units,
                      kernel_size=3, stride=1, padding=1),
            nn.ReLU(),


            nn.Conv2d(hidden_units, hidden_units,
                      kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units,
                      kernel_size=3, stride=1, padding=1),
            nn.ReLU(),


            nn.Conv2d(hidden_units, hidden_units,
                      kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(hidden_units*256, output_features)
        )
    def forward(self, X:pt.Tensor) -> pt.Tensor:
        X_change = self.conv_block_1(X)
        X_change = self.conv_block_2(X_change)
        # print(X_change.shape)
        X_change = self.classifier(X_change)
        return X_change

In [30]:
pt.manual_seed(42)
model0 = ModelWithoutAugmentation(input_features=3, output_features=len(train_dataset.classes), hidden_units=10).to(device)
model0

ModelWithoutAugmentation(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=2560, out_features=3, bias=True)
  )
)

#### Testing model with random data

In [31]:
imgs, labels = next(iter(train_dataloader))
print(imgs.shape, len(labels))
# model0(imgs[0].unsqueeze(0))
model0(imgs.to(device))

torch.Size([32, 3, 64, 64]) 32


tensor([[0.0602, 0.0636, 0.0365],
        [0.0641, 0.0650, 0.0380],
        [0.0633, 0.0635, 0.0402],
        [0.0670, 0.0642, 0.0410],
        [0.0688, 0.0647, 0.0378],
        [0.0595, 0.0594, 0.0380],
        [0.0651, 0.0613, 0.0402],
        [0.0616, 0.0613, 0.0405],
        [0.0687, 0.0646, 0.0417],
        [0.0636, 0.0617, 0.0385],
        [0.0709, 0.0686, 0.0403],
        [0.0679, 0.0707, 0.0401],
        [0.0656, 0.0622, 0.0375],
        [0.0642, 0.0608, 0.0389],
        [0.0605, 0.0604, 0.0399],
        [0.0664, 0.0644, 0.0383],
        [0.0740, 0.0676, 0.0400],
        [0.0639, 0.0624, 0.0379],
        [0.0709, 0.0683, 0.0400],
        [0.0628, 0.0608, 0.0386],
        [0.0647, 0.0643, 0.0413],
        [0.0689, 0.0657, 0.0402],
        [0.0621, 0.0610, 0.0379],
        [0.0655, 0.0618, 0.0404],
        [0.0685, 0.0636, 0.0398],
        [0.0642, 0.0620, 0.0386],
        [0.0630, 0.0637, 0.0382],
        [0.0654, 0.0642, 0.0376],
        [0.0678, 0.0621, 0.0349],
        [0.058

### Summarize a model

In [32]:
try:
    import torchinfo
except ModuleNotFoundError:
    print('Module not found, installing module')
    !pip3 install torchinfo

In [33]:
torchinfo.summary(model0, input_size=[32,3,64,64],device=device)

Layer (type:depth-idx)                   Output Shape              Param #
ModelWithoutAugmentation                 [32, 3]                   --
├─Sequential: 1-1                        [32, 10, 32, 32]          --
│    └─Conv2d: 2-1                       [32, 10, 64, 64]          280
│    └─ReLU: 2-2                         [32, 10, 64, 64]          --
│    └─Conv2d: 2-3                       [32, 10, 64, 64]          910
│    └─ReLU: 2-4                         [32, 10, 64, 64]          --
│    └─MaxPool2d: 2-5                    [32, 10, 32, 32]          --
├─Sequential: 1-2                        [32, 10, 16, 16]          --
│    └─Conv2d: 2-6                       [32, 10, 32, 32]          910
│    └─ReLU: 2-7                         [32, 10, 32, 32]          --
│    └─Conv2d: 2-8                       [32, 10, 32, 32]          910
│    └─ReLU: 2-9                         [32, 10, 32, 32]          --
│    └─MaxPool2d: 2-10                   [32, 10, 16, 16]          --
├─Sequentia

In [39]:
from helper_functions import accuracy_fn
loss_fn = nn.CrossEntropyLoss()

optimizer = pt.optim.SGD(params = model0.parameters(), lr=0.1)

def accuracy_fn(y_logits, y_true:pt.Tensor):
    print(y_logits)
    y_preds = y_logits.argmax(1)
    # train_acc += (y_preds==y_true).sum().item()/len(y_logits)
    train_acc += pt.eq(y_preds,y_true).sum().item()/len(y_logits)

    return train_acc

In [40]:
from ml_funcs import Model_operations, Timer
import tqdm
timer = Timer()
epochs = 3
for epoch in range(epochs):
    timer.interval()
    Model_operations.train_step(model0, train_dataloader, optimizer, loss_fn, accuracy_fn,device,True)
    Model_operations.test_step(model0, test_dataloader, loss_fn, accuracy_fn,device,True)
timer.show_as_print()

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)